# సెమాంటిక్ కర్నెల్

ఈ కోడ్ నమూనాలో, మీరు [సెమాంటిక్ కర్నెల్](https://aka.ms/ai-agents-beginners/semantic-kernel) AI ఫ్రేమ్‌వర్క్‌ను ఉపయోగించి ఒక ప్రాథమిక ఏజెంట్‌ను సృష్టిస్తారు.

ఈ నమూనా యొక్క లక్ష్యం, వివిధ ఏజెంటిక్ నమూనాలను అమలు చేసే అదనపు కోడ్ నమూనాల్లో మేము ఉపయోగించే దశలను మీకు చూపించడం.


## అవసరమైన పైథాన్ ప్యాకేజీలను దిగుమతి చేసుకోండి


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## క్లయింట్‌ను సృష్టించడం

ఈ ఉదాహరణలో, LLM యాక్సెస్ కోసం [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ను ఉపయోగిస్తాము.

`ai_model_id` ను `gpt-4o-mini` గా నిర్వచించారు. GitHub Models మార్కెట్‌ప్లేస్‌లో అందుబాటులో ఉన్న మరో మోడల్‌కు మోడల్‌ను మార్చి వివిధ ఫలితాలను చూడండి.

GitHub Models కోసం `base_url` కోసం ఉపయోగించే `Azure Inference SDK` ను ఉపయోగించడానికి, Semantic Kernel లోని `OpenAIChatCompletion` కనెక్టర్‌ను ఉపయోగిస్తాము. ఇతర మోడల్ ప్రొవైడర్ల కోసం Semantic Kernel ఉపయోగించడానికి [అందుబాటులో ఉన్న ఇతర కనెక్టర్లు](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) కూడా ఉన్నాయి.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ఏజెంట్‌ను సృష్టించడం

కింద, మేము `TravelAgent` అనే ఏజెంట్‌ను సృష్టిస్తున్నాము.

ఈ ఉదాహరణలో, మేము చాలా సులభమైన సూచనలను ఉపయోగిస్తున్నాము. మీరు ఈ సూచనలను మార్చి ఏజెంట్ ఎలా భిన్నంగా స్పందిస్తుందో చూడవచ్చు.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ఏజెంట్ నడపడం

ఇప్పుడు మేము `ChatHistoryAgentThread` అనే రకానికి చెందిన ఒక థ్రెడ్‌ను నిర్వచించడం ద్వారా ఏజెంట్‌ను నడపవచ్చు. ఏదైనా అవసరమైన సిస్టమ్ సందేశాలు ఏజెంట్ యొక్క invoke_stream యొక్క `messages` కీవర్డ్ ఆర్గ్యుమెంట్‌కు అందించబడతాయి.

ఇవి నిర్వచించిన తర్వాత, మేము ఒక `user_inputs` సృష్టిస్తాము, ఇది యూజర్ ఏజెంట్‌కు పంపే సందేశం అవుతుంది. ఈ సందర్భంలో, మేము ఈ సందేశాన్ని `Plan me a sunny vacation` గా సెట్ చేసాము.

ఏజెంట్ ఎలా భిన్నంగా స్పందిస్తుందో చూడటానికి ఈ సందేశాన్ని మార్చడానికి సంకోచించకండి.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమానతలు ఉండవచ్చు. దయచేసి, మూల భాషలో ఉన్న అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించండి. ముఖ్యమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
